# Giving voice to madrigals

This Jupyter Notebook was created for the exam in Electronic Publishing and Digital Storytelling taught by Prof. Marilena Daquino at the University of Bologna for the year 2021-2022.

Our research starts form the dataset of the **British Library about the History of Printed Music**. It is downloadable as a [csv file](https://www.bl.uk/bibliographic/downloads/HistoryOfMusicResearcherFormat_202210_csv.zip).
We then load the csv using the pandas library in order to examine our data.

We then create a Dataframe about madrigals, extracting only rows where the relevant columns contain the characters "madrigal"

<hr>

# 1. Cleaning the British Library dataset

### 1.1 Preliminary cleaning: selecting "madrigals" from the csv

Preliminary installation: googletrans library for the last study about the titles of the text (Uncomment the first line to install the library)<br>Preliminary imports: pandas and numpy for managing the dataset, nltk and re for manipulating the texts in the last section.

In [1]:
#pip install googletrans==3.1.0a0
import pandas as pd
import numpy as np
import nltk
import re
from SPARQLWrapper import SPARQLWrapper, JSON
import ssl

#preparing the connexion to the wikidata endpoint
ssl._create_default_https_context = ssl._create_unverified_context
# get the endpoint API
wikidata_endpoint = "https://query.wikidata.org/bigdata/namespace/wdq/sparql"


In [2]:
#insert here the link to the British Library data dump, after dowloading it.
df = pd.read_csv("C:/Users/const/Downloads/MusicResearcherFormat_201505_csv/detailedrecords.csv",dtype=str)
#selecting only rows containing the word "madrigal"
df_madri = df.loc[df["Subject/genre terms"].str.contains('madrigal', case=False, na=False) | df["Title"].str.contains('madrigal', case=False, na=False) | df["Other titles"].str.contains('madrigal', case=False, na=False) | df["Notes"].str.contains('madrigal', case=False, na=False)]
df_madri.describe()

,BL record ID,Composer,Composer life dates,Title,Standardised title,Other titles,Other names,Publication date (standardised),Publication date (not standardised),Country of publication,...,Contents,Referenced in,Subject/genre terms,Physical description,Series title,Number within series,ISBN,ISMN,Publisher number,BL shelfmark
count,3416,3199,2173,3416,1398,186,1321,3359,3336,2657,...,295,152,629,3262,1376,716,206,64,523,3408
unique,3416,1128,503,3264,759,146,788,320,612,29,...,292,152,262,953,853,664,201,60,507,2766
top,014976870,"Marenzio, Luca",1553-1599,The Chester books of madrigals,Madrigals,Triumphes of Oriana,"Oliphant, Thomas",1980,1980,England,...,Contents: Come shepheard swaynes -- Flourish y...,RISM B/I 1596⁷,"Madrigals, Italian",8°,Das Chorwerk,24 [Corpus Mensurabilis Musicae],1888471158,501002252,CF249,F.280.b
freq,1,115,115,8,38,24,41,51,36,1295,...,2,1,95,662,24,12,4,2,3,95


This dataset contains all rows containg the word madrigal, but for our study we want only those composed by composers in our period of interest (1530 - 1650), so we decided to clean the "Composer life dates" column, adding manually those that are missing. 
<hr>

### 1.2 Manually cleaning the composers dates

We first extract only the column "Composer" and "Composer life dates" and select composers born after 1450 and dead before 1730 (to be sure to involve all composers of interest).
We manually added and corrected the dates, because most of these composers are not present on Wikidata and had to be searched on specialized websites and music dictionaries.

In [3]:
composers_to_search=df_madri[["Composer", "Composer life dates"]]
composers_to_search =composers_to_search.sort_values(["Composer"]).drop_duplicates()
composers_to_search.to_csv("data/composers_to_clean.csv")

After exporting our csv and correcting it manually, we import it back into our project. 

In [4]:
composers = pd.read_csv("data/composers_dates_cleaned.csv")

#dropping lines where we don't have information about life and death dates. 
#(we cannot include them in our final selection of Composers belonging to our period of time)
composers = composers.dropna()
composers.drop('Unnamed: 0', axis=1, inplace=True)
composers.head()

,Composer,Composer life dates
1,A. L. (Amelia Lehmann),1838-1903
2,"Adler, Samuel",1928-
3,"Adriaensen, Emanuel",1550-1604
4,"Adriani, Francesco",1539-1575
5,"Adriansen, Emanuel",1554-1604


<hr>
<a name="step1.3"></a>

### 1.3 Merging the cleaned composers with the records dataframe


Then we select only composers born in the right period: after the first composer of madrigals Philippe Verdelot in 1480, and dead before the last composer of madrigals of our period of interest: Scarlatti in 1725. To include cases of composers born at the extremities of this time frame, we delimited the time period between 1450 and 1730.

In [5]:
dict_composers = composers.set_index("Composer").to_dict()
comp_madrigals = {}

#Selecting only composers born after 1450 and dead before 1730
for composer in dict_composers['Composer life dates']:
    dates = dict_composers['Composer life dates'][composer]
    dates = dates.strip("- ")
    dates_list = dates.split("-")
    if len(dates_list)>1:
        if int(dates_list[0])>=1450 and int(dates_list[1])<=1730:
            comp_madrigals[composer]=dates
    else:
        if int(dates_list[0])>=1450 and int(dates_list[0])<=1730:
            comp_madrigals[composer]=dates
            

#We transform the dictionary into a dataframe and merge it to the one with all instances of works,
#thus obtaining a dataframe where we have the life dates for only composers of interest, and removing useless columns
df_comp_madri = pd.DataFrame.from_dict(comp_madrigals, orient="index")
df_comp_madri = df_comp_madri.reset_index()

#merging the two dataframes
df_comp_madri.columns=["Composer", "Composer life dates"]
df_madri=df_madri.drop(["Composer life dates"], axis=1)
df_madri = df_madri.merge(df_comp_madri, how="left", on="Composer")

#selecting only columns of interest to us
df_madri = df_madri[["BL record ID", "Title", "Standardised title", "Other titles", "Composer", "Composer life dates", "Other names", "Publication date (standardised)", "Country of publication", "Place of publication", "Publisher", "Notes", "Contents","Subject/genre terms"]]
df_madri.describe()

,BL record ID,Title,Standardised title,Other titles,Composer,Composer life dates,Other names,Publication date (standardised),Country of publication,Place of publication,Publisher,Notes,Contents,Subject/genre terms
count,3416,3416,1398,186,3199,1843,1321,3359,2657,2681,2782,1020,295,629
unique,3416,3264,759,146,1128,324,788,320,29,310,866,962,292,262
top,014976870,The Chester books of madrigals,Madrigals,Triumphes of Oriana,"Marenzio, Luca",1553-1599,"Oliphant, Thomas",1980,England,London,Novello & Co,Score has biography of the composer and editor...,Contents: Come shepheard swaynes -- Flourish y...,"Madrigals, Italian"
freq,1,8,38,24,115,115,41,51,1295,1018,181,6,2,95


<hr>


### 1.4 Extracting madrigals published in the period of interest
We filter the data on publication date to try to clean data and extract only relevant madrigals.

By examining our dataset, we noticed that there were a few more entries for madrigals until the end of the 1600s (until 1678) before a jump to 1762 where the genre of madrigals changes. So we decided to include these in the final dataset

In [6]:
#create a dataset focusing on the publication date
df_madri_pub_bef_1678 = df_madri.copy(deep=True)

#drop records where we don't have the publication date
df_madri_pub_bef_1678 = df_madri_pub_bef_1678.dropna(subset=['Publication date (standardised)'])

#selecting only records published before 1678
df_madri_pub_bef_1678['Publication date (standardised)'] = df_madri_pub_bef_1678['Publication date (standardised)'].astype("int")
df_madri_pub_bef_1678 = df_madri_pub_bef_1678.loc[df_madri_pub_bef_1678['Publication date (standardised)'] <= 1678]

<hr>

### 1.5 Merging madrigals composed by 15th-16th century composers with madrigals published before 1678
From the dataframe containing all records of madrigals, we drop the ones without life dates for the composers (those out of our period of interest) and then merge the madrigals composed 1678 extracted at the last step.

Our dataset of madrigals will then contain madrigals from composers between 1450 and 1730 and/or composed before 1678

In [7]:
df_madri_pub_bef_1678['Publication date (standardised)'] = df_madri_pub_bef_1678['Publication date (standardised)'].astype("str")

#We want to keep only columns where we registered dates for a composer -> those we know are of the right period of time.
df_madri_composers_dates=df_madri.dropna(subset=['Composer life dates'])

#Then merge records which have been published before 1678, even though we don't have dates for the composers
df_madri = df_madri_composers_dates.merge(df_madri_pub_bef_1678, how="outer")

df_madri.describe()

,BL record ID,Title,Standardised title,Other titles,Composer,Composer life dates,Other names,Publication date (standardised),Country of publication,Place of publication,Publisher,Notes,Contents,Subject/genre terms
count,1963,1963,1218,149,1905,1843,1000,1928,1501,1507,1623,654,236,458
unique,1963,1864,624,115,404,324,562,274,20,209,518,607,234,165
top,014963847,Opera omnia,Madrigals,Triumphes of Oriana,"Marenzio, Luca",1553-1599,"Oliphant, Thomas",1980,Italy,London,Appresso Angelo Gardano,Score has biography of the composer and editor...,Dunque fia ver dicea che mi convegna (seconda ...,"Madrigals, Italian"
freq,1,8,34,23,115,115,41,36,613,364,127,6,2,92


In [8]:
df_madri.to_csv("data/madrigals_records_to_clean.csv")

<hr>

### 1.6 Cleaning the dataset by hand, adding the column "Voices" and "Titles madrigals"
By hand, we extracted the number of voices for which each record has been composed, as well as a new column for extracting the title of the single madrigal, if it applies. Due to the fact that the records involve several languages and the dataset was messy, it was easier to extract these information manually than to use computation.

In [11]:
records_cleaned = pd.read_csv("data/madrigals_records_cleaned.csv", sep=";", encoding="utf-8")
records_cleaned.head()

,BL record ID,Title,Standardised title,Other titles,Composer,Composer life dates,Other names,Publication date (standardised),Country of publication,Place of publication,Publisher,Notes,Contents,Subject/genre terms,Voices,Titles madrigals
0,4166293.0,Il Primo Libro de Madrigali a Cinque Voci ... ...,NaN,NaN,"Adriani, Francesco",1539-1575,NaN,1570.0,Italy,Vinegia,Appresso Girolamo Scotto,NaN,NaN,NaN,5,NaN
1,4166294.0,Il Secondo Libro de Madrigali a Cinque Voci .....,NaN,NaN,"Adriani, Francesco",1539-1575,NaN,1570.0,Italy,Vinegia,Appresso Girolamo Scotto,NaN,NaN,NaN,5,NaN
2,4166295.0,"Luitmuziek ... Een keuze van fantasieën, danse...",NaN,NaN,"Adriansen, Emanuel",1554-1604,"Spiessons, Godelieve",1966.0,Belgium,Antwerpen,Vereniging voor musiekgeschiedenis te Antwerpen,NaN,NaN,NaN,4; 5; 6,NaN
3,4166296.0,Novvm pratvm mvsicvm longo amoenissimvm : cviv...,Novum pratum musicum,Novum pratum musicum longo amoenissimum,"Adriaensen, Emanuel",1550-1604,"Phalèse, Pierre [printer] ; Bellère, Pierre [p...",1592.0,Belgium,Antverpiæ ; Antwerp,Excudebat Petrus Phalesius sibi & Ioanni Bellero,Method in lute tabulature followed by solo lut...,Methodvs ad omnes omnivm tonorvm cantiones in ...,Lute--Methods ; Lute music ; Madrigals ; Part ...,4; 5; 6,NaN
4,4166297.0,Pratum Musicum ... cuius ambitu ... comprehend...,NaN,NaN,"Adriansen, Emanuel",1554-1604,NaN,1600.0,Belgium,Antuerpiæ,Ex Typographia Musica Petri Phalesij,The contents of this edition are different fro...,NaN,NaN,4; 5; 6,NaN


<hr>

# 2. Working on composers of interest: 

### 2.1 Reformatting the names
We want to merge our dictionary of composers instantiated at [step 1.3](#step1.3) with information extracted from Wikidata.
First we need to change the name in format "surname, name" to "name surname" which is the one adopted for labels on Wikidata.

In [12]:
dict_comp_madrigals = {}

#this dictionary will be used later on when working with the publishers and records dataframes
inverted_names = {}

for el in comp_madrigals.keys():
    
    #exchanging "surname, name" for "name surname"
    if len(el.split(", "))>1:
        splitted = el.split(", ")
        name = splitted[1].strip("'")
        surname = splitted[0].strip("'")
        dict_comp_madrigals[name+" "+surname] = comp_madrigals[el]
        inverted_names[el] = name+" "+surname
        
    #some entities, like "Henry VIII", are not in "surname, name" format, so they should be added as is.
    else:
        dict_comp_madrigals[el] = comp_madrigals[el]
        
#printing the length of the dictionary        
print(len(dict_comp_madrigals))

369


<hr>

### 2.2 Querying the Wikidata SPARQL endpoint API
When sending a query to the wikidata sparql endpoint api, the full list was too big to handle, so we split it in groups of 100 composers.

We use regular expressions to search for persons which have as occupation "composer" and whose label matches one in the list. 
By groups of 100, the names are concatenated in a big string with the operator OR ("|") and we make sure that the name starts ("^") and ends ("$") exactly with those characters.

Without this condition, the first query returned other composers which were almost homonyms from the ones in our list, for example "Friedrich Nicolaus Bruhns" instead of only "Nicolaus Bruhns".

In [13]:
#one query with all the composers is too long, so we split the list in sub_lists of 100 composers.
list_keys = list(dict_comp_madrigals.keys())
chunked_list_to_be_queried = [list_keys[i:i+100] for i in range(0, len(dict_comp_madrigals), 100)]
#instantiating the final dictionary
dict_composers_wd = {}

In [14]:
#sending a query for groups of 100 composers
for hundred_composers in chunked_list_to_be_queried:
    to_str = "$|^".join(hundred_composers)

    query_composers = """
    SELECT DISTINCT *
    WHERE {
            ?composer wdt:P106 wd:Q36834; #has for occupation: composer
            rdfs:label ?label.
            FILTER regex(?label, \"^"""+to_str+"""$\" )
            FILTER (langMatches(lang(?label), "EN"))
            }
    """
    # set the endpoint 
    sparql_wd = SPARQLWrapper(wikidata_endpoint)
    # set the query
    sparql_wd.setQuery(query_composers)
    # set the returned format
    sparql_wd.setReturnFormat(JSON)
    # get the results
    results = sparql_wd.query().convert()

    # manipulate the result
    for result in results["results"]["bindings"]:
        #the dictionary contains as key the name of the composers, 
        #and as value another dictionary containing the future columns of the dataframe, "wikidata" and "dates"
        dict_composers_wd[result["label"]["value"]] = {"wikidata":result["composer"]["value"], 
                                                       "dates":dict_comp_madrigals[result["label"]["value"]]}

#printing the length of the dictionary
print(len(dict_composers_wd))

244


We get a dataset of 244 composers out of the 369 in the British Library catalogue, so approximately 2/3 of them.

We now send another query to wikidata to see if we can find other composers of madrigals to add to our dataset.

In [15]:
query_new_composers = """
SELECT ?composition ?compositionLabel ?composer ?composerLabel ?birthdate ?deathdate WHERE {
        ?composition wdt:P86 ?composer;         #a piece composed by someone, 
                     wdt:P7937 wd:Q193217.      #and is a form of creative work of a madrigal.
        ?composer wdt:P569 ?birthdate;
                  wdt:P570 ?deathdate.
          SERVICE wikibase:label { bd:serviceParam wikibase:language "en" }  # labels in English
}

"""

# set the endpoint 
sparql_wd = SPARQLWrapper(wikidata_endpoint)
# set the query
sparql_wd.setQuery(query_new_composers)
# set the returned format
sparql_wd.setReturnFormat(JSON)
# get the results
results = sparql_wd.query().convert()

# manipulate the result
for result in results["results"]["bindings"]:
    #Adding the composer only if it doesn't already exist in the dataset
    if result["composerLabel"]["value"] not in dict_composers_wd:
        birthdate = result["birthdate"]["value"].split("-")
        deathdate = result["deathdate"]["value"].split("-")
        dict_composers_wd[result["composerLabel"]["value"]] = {"wikidata":result["composer"]["value"],
                                                              "dates":birthdate[0]+"-"+deathdate[0]}

        
#printing the length of the dictionary
print(len(dict_composers_wd))

245


This allowed to add only one composer to our collection. Indeed, Wikidata has very few instances of madrigals registered.

We tried to query the endpoint to find more information, for example setting madrigals as the movement (P135) or the genre (P136) to which the composer belonged, but none was conclusive.

So we remain with our dataset of 245 individuals.
<hr>

### 2.3 Adding metadata for our composers via the SPARQL endpoint API
Now let's add some information: Nationality, gender, birthplace, place of death, languages spoken and instruments played.

In [16]:
#creating a dictionary for mapping the output of wikidata with the future columns of the dataframe
labels = {"genderLabel":"gender", "citizenshipLabel":"citizenship", "birthplaceLabel":"birthplace", "birthcountryLabel":"birth country", "geoBirthplace":"geographical coordinates birthplace", "deathplaceLabel":"place of death", "deathcountryLabel":"country of death", "geoDeathplace":"geographical coordinates place of death", "languageLabel":"language", "instrumentLabel":"instrument"}

In [17]:
for composer in dict_composers_wd:
    #collecting all information available on wikidata about each composer
    query_composer_info = """
    SELECT ?genderLabel ?citizenshipLabel ?birthplaceLabel ?birthcountryLabel ?geoBirthplace ?deathplaceLabel ?deathcountryLabel ?geoDeathplace ?languageLabel ?instrumentLabel WHERE {
           OPTIONAL {<""" +dict_composers_wd[composer]['wikidata']+"""> wdt:P21 ?birth_date } .
           OPTIONAL {<""" +dict_composers_wd[composer]['wikidata']+"""> wdt:P21 ?gender } .
           OPTIONAL {<""" +dict_composers_wd[composer]['wikidata']+"""> wdt:P27 ?citizenship } .
           OPTIONAL {<""" +dict_composers_wd[composer]['wikidata']+"""> wdt:P19 ?birthplace .
                       ?birthplace wdt:P17 ?birthcountry;
                                   wdt:P625 ?geoBirthplace} .
           OPTIONAL {<""" +dict_composers_wd[composer]['wikidata']+"""> wdt:P20 ?deathplace .
                       ?deathplace wdt:P17 ?deathcountry;
                                   wdt:P625 ?geoDeathplace} .
           OPTIONAL {<""" +dict_composers_wd[composer]['wikidata']+"""> wdt:P1412 ?language } .
           OPTIONAL {<""" +dict_composers_wd[composer]['wikidata']+"""> wdt:P1303 ?instrument } .
              SERVICE wikibase:label { bd:serviceParam wikibase:language "en" }  # labels in English
    }

    """
    # set the endpoint 
    sparql_wd = SPARQLWrapper(wikidata_endpoint)
    # set the query
    sparql_wd.setQuery(query_composer_info)
    # set the returned format
    sparql_wd.setReturnFormat(JSON)
    # get the results
    results = sparql_wd.query().convert()

    # manipulate the result
    for result in results["results"]["bindings"]:
        for label in labels:
            if label in result:
                #a composer might have more than one citizenship, language or instrument, and in that case we create a list.
                if labels[label] in ["citizenship", "language", "instrument"]:
                    if labels[label] in dict_composers_wd[composer] and result[label]["value"] not in dict_composers_wd[composer][labels[label]]:
                        if type(dict_composers_wd[composer][labels[label]]) != list:
                            dict_composers_wd[composer][labels[label]] = list([dict_composers_wd[composer][labels[label]]])
                        dict_composers_wd[composer][labels[label]].append(result[label]["value"])
                    elif labels[label] not in dict_composers_wd[composer]:
                        dict_composers_wd[composer][labels[label]] = result[label]["value"]
                        
                #otherwise, just adding the result to the dictionary.
                else:
                    dict_composers_wd[composer][labels[label]] = result[label]["value"]

In [18]:
#create a dataframe from the dictionary obtained
df_composers = pd.DataFrame.from_dict(dict_composers_wd, orient="index")
df_composers.reset_index(inplace=True)
df_composers = df_composers.rename(columns = {'index':'name'})
df_composers.describe()

,name,wikidata,dates,gender,citizenship,birthplace,birth country,geographical coordinates birthplace,place of death,country of death,geographical coordinates place of death,language,instrument
count,245,245,245,240,130,178,178,178,168,168,168,112,84
unique,245,245,235,2,33,113,11,112,82,15,82,14,22
top,Giovanni Maria Nanino,http://www.wikidata.org/entity/Q3071924,1544-1590,male,Republic of Venice,Florence,Italy,Point(11.342777777 44.493888888),Rome,Italy,Point(12.482777777 41.893055555),Italian,organ
freq,1,1,2,237,29,8,114,8,17,104,17,47,35


We save the final dataset in json and csv in order to use it for visualizations later on.

In [16]:
df_composers.to_json("data/final datasets/composers.json")
df_composers.to_csv("data/final datasets/composers.csv")

<hr>

# 3. Working on publishers
In order to do a study and visualizations on the publishers, we need to uniformize the data. For this section, we work only with madrigals with a publication date registered before 1678.

### 3.1 Cleaning publication place

From the column "Place of publication", we find the standard name in english and replace the name. 

Due to the different orthographs of the names, as well as deprecated names which are not present on Wikidata, it was simpler to write by hand a dictionary to map the old names to the new ones, by searching for them on Google, rather than writing a SPARQL query to extract this information. 

In [19]:
places = df_madri_pub_bef_1678[["Place of publication"]]
set_places = set(places["Place of publication"].dropna())  
print(sorted(set_places))

['A Paris', 'Antuerpiæ', 'Antverpiæ ; Antwerp', 'Anuers', 'Anuersa', 'Anuersa ; Venice', 'Anversa', 'At London', 'Augspurg', 'Augusta', 'Bologna', 'Coloniae Agrippinae ; Cologne', 'Copenhave', 'Copenhaven', 'Copenhavē', 'Dresdae ; Dresden', 'En Anvers ; Antwerp', 'Erffurt', 'Excudebat Venetiis', 'Excudebat Venetiis ; Venice', 'Ferrara', 'Firenze', 'Firenze ; Florence', 'Francfort', 'Francoforti', 'Freybergk in Meissen', 'Genova', 'Getruckt zu Strassburg ; Strasbourg', 'Getruckt zů Strassburg ; Strasbourg', 'Gotha', 'Heidelberg', 'In Anuersa', 'In Anuersa ; Antwerp', 'In Anversa ; Antwerp', 'In Ferrara', 'In Roma ; Rome', 'In Roma ; Venice', 'In Venetia', 'In Venetia ; Venice', 'In Vineggia ; Venice', 'In Vinegia ; Venice', 'In Vinetia ; Venice', 'Jehna', 'Leipzig', 'London', 'Londra', 'Lyon', 'Lyone', 'Messina', 'Milano', 'Milano ; Venice', 'Monachii ; Munich', 'Monachij', 'Napoli', 'Noribergae', 'Noribergae ; Nuremberg', 'Noribergæ', 'Norimbergæ', 'Nürmberg', 'Nürnberg', 'Oruieto', 'O

In [20]:
dict_places = {'Veneggia': 'Venice', 'Excudebat Venetiis': 'Venice', 'Venetia ; Venice':'Venice', 'Noribergæ':'Nuremberg', 'Anuersa ; Venice':'Antwerp', 'Londra':'London', 
              'Gotha':'Gotha', 'Milano':'Milan', 'Augspurg':'Augsburg', 'Copenhavē':'Copenhagen', 'In Vinegia ; Venice':'Venice', 'In Roma ; Rome':'Rome', 'Anversa':'Antwerp',
              'Padua':'Padua', 'Noribergae':'Nuremberg', 'Anuersa':'Antwerp', 'Excudebat Venetiis ; Venice':'Venice', 'Dresdae ; Dresden': 'Dresden', 'Copenhave':'Copenhagen',
              'Oruieto':'Orvieto', 'Venetijs ; Venice':'Venice', 'In Venetia ; Venice':'Venice', 'In Vinetia ; Venice':'Venice', 'Bologna':'Bologna', 'Nürmberg':'Nuremberg',
              'Ventia':'Venice', 'Vinegia':'Venice', 'Firenze ; Florence':'Florence', 'Venetia':'Venice', 'Freybergk in Meissen':'Freiberg', 'Venegia':'Venice',
              'Getruckt zů Strassburg ; Strasbourg':'Strasbourg', 'Vinetia':'Venice', 'Leipzig':'Leipzig', 'Venetijs':'Venice', 'Ferrara':'Ferrara', 'Palermo':'Palermo',
              'Venetiis':'Venice', 'In Venetia':'Venice', 'In Anuersa':'Antwerp', 'Venezia':'Venice', 'Nürnberg':'Nuremberg', 'Francoforti':'Frankfurt am Main', 'Erffurt':'Erfurt',
              'Parma':'Parma', 'Anuers':'Antwerp', 'Venetiis ; Venice':'Venice', 'Oxford':'Oxford', 'Jehna':'Jena', 'Antverpiæ ; Antwerp':'Antwerp', 'Vineggia':'Venice',
              'Firenze':'Florence', 'Antuerpiæ':'Antwerp', 'in Anversa':'Antwerp', 'In Vineggia ; Venice':'Venice', 'London':'London', 'Napoli':'Naples', 'Augusta':'Augsburg',
              'Heidelberg':'Heidelberg', 'Wolferbyti':'Wolfenbüttel', 'Genova':'Genoa', 'Copenhaven':'Copenhagen', 'En Anvers ; Antwerp':'Antwerp', 'Roma':'Rome', 'Rotterodamo':'Rotterdam',
              'Venice':'Venice', 'Monachij':'Munich','In Roma ; Venice':'Rome', 'Stampato in Ferrara, et ristampato in Napoli, Per Constantino Vitale Ad istanza di Stefano Colacurcio':'Ferrara',
              'A Paris':'Paris', 'At London':'London', 'Coloniae Agrippinae ; Cologne':'Cologne', 'Francfort':'Frankfurt am Main', 'Getruckt zu Strassburg ; Strasbourg':'Strasbourg',
              'In Anuersa ; Antwerp':'Antwerp', 'In Anversa ; Antwerp':'Antwerp', 'In Ferrara':'Ferrara', 'Lyon':'Lyon', 'Lyone':'Lyon', 'Messina':'Messina','Milano ; Venice':'Milan',
              'Monachii ; Munich':'Munich', 'Noribergae ; Nuremberg':'Nuremberg', 'Norimbergæ':'Nuremberg','Pataviæ':'Padua', 'Rotenburg ob der Tauber':'Rothenburg ob der Tauber ',
              'Vineggia ; Venice':'Venice'}

In [21]:
#replacing the cleaned names in the dataframe
df_madri_pub_bef_1678= df_madri_pub_bef_1678.replace({"Place of publication":dict_places})

#We realized that the data was wrong: the country of publication for the place Augsburg should be "Germany" and not "United States"
df_madri_pub_bef_1678 = df_madri_pub_bef_1678.replace({'Country of publication':{'United States':'Germany (East)'}})

<hr>

### 3.2 Cleaning publisher column
The orthograph of names being fluctuant at that time, there were several ways of writing the name of the same publisher, so we did a pre-cleaning with regular expressions, but then a better cleaning by hand in order to make sure that all rows published by the same publisher could be identified.

In [22]:
df_madri_pub_bef_1678= df_madri_pub_bef_1678.replace({"Composer":inverted_names})
df_madri_pub_bef_1678['Publisher'] = df_madri_pub_bef_1678['Publisher'].replace(to_replace ='[aA](p)?presso |[aA]pud |[pP]resso ', value = '', regex = True)
df_madri_pub_bef_1678.to_csv("data/publishers_to_clean.csv")

We continued the cleaning by hand to uniformize the names, and then merge the resulting dataframe with the new information about voices and madrigal title manually added to the records_cleaned dataframe.

In [23]:
publishers_cleaned = pd.read_csv("data/publishers_cleaned.csv", index_col="Column1")
publishers_cleaned = publishers_cleaned.merge(records_cleaned[['BL record ID', 'Voices', 'Titles madrigals']], how="left", on=["BL record ID"])

#with regular expressions, removing the "East" of "East Germany" registered for some records
publishers_cleaned['Country of publication'] = publishers_cleaned['Country of publication'].replace(to_replace =' \(East\)', value = '', regex = True)

<hr>

### 3.3 Adding geographical coordinates
We realized that in order to compute geographical visualizations for the publishers, we could extract coordinates for publishers location from Wikidata, first by creating a dictionary of cities and their country extracted from the dataframe.

In [24]:
cities = dict(zip(publishers_cleaned['Place of publication'].dropna(),publishers_cleaned['Country of publication'].dropna()))

#it was necessary to change 'England' to 'United Kingdom' for the Wikidata query 
#which wouldn't have extracted Oxford and London otherwise
for city in cities: 
    if cities[city] == "England":
        cities[city] = "United Kingdom"

In [25]:
cities_locations={}
for city in cities:
    #query the endpoint based on the labels of the cities and countries.
    query_geolocations = """
    SELECT DISTINCT ?geolocation WHERE {
            {?city wdt:P31/wdt:P279* wd:Q515.} #instance or subclass of a city
            UNION
            {?city wdt:P31 wd:Q747074.} #instance of a comune of Italy (to include Orvieto which is not a city)
            ?city wdt:P17 ?country.
            ?country rdfs:label \""""+cities[city]+"""\"@en.
            ?city wdt:P625 ?geolocation.
            ?city rdfs:label \""""+city+"""\"@en.
            SERVICE wikibase:label { bd:serviceParam wikibase:language "en" }
        }

    """

    # set the endpoint 
    sparql_wd = SPARQLWrapper(wikidata_endpoint)
    # set the query
    sparql_wd.setQuery(query_geolocations)
    # set the returned format
    sparql_wd.setReturnFormat(JSON)
    # get the results
    results = sparql_wd.query().convert()

    # manipulate the result
    for result in results["results"]["bindings"]:
        cities_locations[city] = result['geolocation']['value']

In [26]:
#create a new column for the geolocation of the place of publication
new_column = publishers_cleaned['Place of publication'].apply(lambda x: (cities_locations[x] if x in cities_locations else np.nan))
publishers_cleaned.insert(loc = 10,
          column = 'Coordinates place of publication',
          value = new_column)
publishers_cleaned.to_csv('data/final datasets/final_publishers_cleaned.csv')
publishers_cleaned.head()

,BL record ID,Title,Standardised title,Other titles,Composer,Composer life dates,Other names,Publication date (standardised),Country of publication,Place of publication,Coordinates place of publication,Publisher,Notes,Contents,Subject/genre terms,Voices,Titles madrigals
0,4166293,Il Primo Libro de Madrigali a Cinque Voci ... ...,NaN,NaN,Francesco Adriani,1539-1575,NaN,1570,Italy,Venice,Point(12.331944444 45.439722222),Girolamo Scotto,NaN,NaN,NaN,5,NaN
1,4166294,Il Secondo Libro de Madrigali a Cinque Voci .....,NaN,NaN,Francesco Adriani,1539-1575,NaN,1570,Italy,Venice,Point(12.331944444 45.439722222),Girolamo Scotto,NaN,NaN,NaN,5,NaN
2,4166296,Novvm pratvm mvsicvm longo amoenissimvm : cviv...,Novum pratum musicum,Novum pratum musicum longo amoenissimum,Emanuel Adriaensen,1550-1604,"Phalèse, Pierre [printer] ; Bellère, Pierre [p...",1592,Belgium,Antwerp,Point(4.399722222 51.221111111),Pietro Phalesio & Giouanni Bellero,Method in lute tabulature followed by solo lut...,Methodvs ad omnes omnivm tonorvm cantiones in ...,Lute--Methods ; Lute music ; Madrigals ; Part ...,4; 5; 6,NaN
3,4166297,Pratum Musicum ... cuius ambitu ... comprehend...,NaN,NaN,Emanuel Adriansen,1554-1604,NaN,1600,Belgium,Antwerp,Point(4.399722222 51.221111111),Pietro Phalesio,The contents of this edition are different fro...,NaN,NaN,4; 5; 6,NaN
4,4166441,Di Agostino Agazzari ... Il Primo Libro de Mad...,NaN,NaN,Agostino Agazzari,1578-1640,NaN,1600,Italy,Venice,Point(12.331944444 45.439722222),Angelo Gardano,NaN,NaN,NaN,5; 6; 8,NaN


<hr>

# 4. Study on madrigal texts

Creating a new dataframe with single titles of madrigals (when they are present in the dataset).

We want to finish our study by examining the texts of madrigals, or rather the titles since the full texts were not available in the dataset. The titles represent the first words of the text, so they already constitute an excerpt of madrigal texts. We want to study if there are recurring themes we can see emerge from our dataset.

In order to include as much madrigal titles as possible, we involved also those published after 1768, as long as the composer was of the right period of time, that have been edited afterwards but still carry the music and lyrics of a 16th-17th century madrigal. 

In [27]:
#preliminary imports
from nltk.corpus import stopwords
en_stops = set(stopwords.words('english'))
fr_stops = set(stopwords.words('french'))
it_stops = set(stopwords.words('italian'))
ge_stops = set(stopwords.words('german'))
all_stops = en_stops.union(fr_stops, it_stops, ge_stops)

from googletrans import Translator, constants
translator = Translator()

<hr>

### 4.1 Selecting rows where the title(s) of madrigal(s) is present

We select the records with information about the titles, either when it has been as a single madrigal, or when the column "Contents" has the list of madrigals present in the collection

In [28]:
#selecting records which have a single madrigal in the title.
records_cleaned = records_cleaned.replace({"Composer":inverted_names})
madrigals_cleaned = records_cleaned.loc[records_cleaned['Titles madrigals'].notnull()]
madrigals_cleaned = madrigals_cleaned.drop_duplicates(['Titles madrigals','Composer'])

#adding collections which have the list of madrigals in the column 'Contents'
madrigals_cleaned = madrigals_cleaned.append(records_cleaned.loc[records_cleaned['Contents'].notnull()])
print(len(madrigals_cleaned))

795


<hr>

### 4.2 Extracting keywords 
For each title, we extract relevant words, by splitting the title and excluding stopwords ("the", "a", etc)

In [29]:
dictionary_madrigals={}
idx = 0
for idx_row,row in madrigals_cleaned.iterrows():
    if not pd.isna(row['Titles madrigals']):
        madrigals = row['Titles madrigals'].split(';')
        for madrigal in madrigals: 
            translations = madrigal.split('. ')
            for translation in translations:
                words = re.split(r"[ :\-!,.]+|'s", translation)
                list_keywords = []
                for word in words: 
                    if word.lower() not in all_stops and word !="":
                        list_keywords.append(word.lower().strip(" '"))
                dictionary_madrigals[idx]={'Madrigal':translation, 'Composer single madrigal':row['Composer'], 'Voices':row['Voices'],'Keywords':list_keywords}
                idx+=1
                break
    #when the title of the madrigal is in the Contents column
    else:
        list_of_madrigals = re.split(r" -+ |;\s*", row['Contents'])
        for madrigal in list_of_madrigals:
            composer = np.nan
            madri = madrigal
            if "/" in madrigal:
                temp = re.split(r"\s*/\s*", madrigal)
                madri = temp[0]
                composer = temp[1].strip("[] ")
            words = re.split(r"[ :\-!,.]+|'s", madri)
            list_keywords = []
            for word in words: 
                if word.lower() not in all_stops and word !="":
                    list_keywords.append(word.lower())
            dictionary_madrigals[idx]={'Madrigal':madri, 'Composer single madrigal':composer, 'Composer collection':row['Composer'], 
                                        'Voices':row['Voices'], 'Keywords': list_keywords}
            idx+=1
    
new_df_by_madrigal = pd.DataFrame.from_dict(dictionary_madrigals, orient="index")  

Since there is a mix of several languages unequally distributed in the dataset, in order to search for recurring themes in madrigals, regardless of the language or translation of the original title, we decided to add a column containing the keywords in english only. 
<hr>

### 4.3 Adding keywords in english using the Google translate API

**Warning: the next cell takes several minutes to run**

It sends a high number of request to the Google translate API.
For each title, we use google translate to get the english translation and then apply the same function as before to remove stopwords and obtain a list of keywords.

In [30]:
#Adding keywords in english
new_df_by_madrigal['Keywords_en'] = new_df_by_madrigal['Madrigal'].apply(lambda x: [word.lower().strip(" '") for word in re.split(r"[ :\-!,.]+|'s", translator.translate(x).text) if word.lower() not in all_stops and word !=""])

In [31]:
new_df_by_madrigal

,Madrigal,Composer single madrigal,Voices,Keywords,Composer collection,Keywords_en
0,Shall I abide this jesting,Richard Alison,5,"[shall, abide, jesting]",NaN,"[shall, abide, jesting]"
1,A garden is my lady's face.,Richard Alison,5,"[garden, lady, face]",NaN,"[garden, lady, face]"
2,There is a Garden in her Face,Richard Alison,5,"[garden, face]",NaN,"[garden, face]"
3,Ah me! Where is my true Love,Felice Anerio,4,"[ah, true, love]",NaN,"[ah, true, love]"
4,"When lo, by Break of Morning",Felice Anerio,4,"[break, morning]",NaN,"[break, morning]"
...,...,...,...,...,...,...
6150,The Nightingale that sweetly doth complayne (s...,Peter Phillips,5,"[nightingale, sweetly, doth, complayne, (secon...",NaN,"[nightingale, sweetly, doth, complayne, (secon..."
6151,As Mopsus went his silly flock foorth leading,Stefano Venturi,5,"[mopsus, went, silly, flock, foorth, leading]",NaN,"[mopsus, went, silly, flock, foorth, leading]"
6152,Flora faire Nimph whilst silly Lambs are feeding,Giovanni Feretti,5,"[flora, faire, nimph, whilst, silly, lambs, fe...",NaN,"[flora, fairy, nymph, silly, lambs, feeding]"
6153,"My sweet Layis, Lady mistres",Giovanni di Macque,5,"[sweet, layis, lady, mistres]",NaN,"[sweet, layis, lady, mistress]"


In [32]:
#saving the final dataset
new_df_by_madrigal.to_csv("data/final datasets/madrigals_by_title.csv")

In [37]:
test_word_cloud = pd.read_csv("madrigals_by_title.csv")

In [40]:
"""big_string=""
for idx_row,row in new_df_by_madrigal.iterrows():
    big_string+=" "+" ".join(row['Keywords'])
for word in ['part', 'first', 'second', 'third', 'fourth', 'fifth', 'sixth','prima', 'seconda', 'parte', 'terza', 'quarta']:
    big_string = big_string.replace(word, "")
#print(big_string)
"""

'big_string=""\nfor idx_row,row in new_df_by_madrigal.iterrows():\n    big_string+=" "+" ".join(row[\'Keywords\'])\nfor word in [\'part\', \'first\', \'second\', \'third\', \'fourth\', \'fifth\', \'sixth\',\'prima\', \'seconda\', \'parte\', \'terza\', \'quarta\']:\n    big_string = big_string.replace(word, "")\n#print(big_string)\n'

In [38]:
composers_cities = pd.read_csv("data/final datasets/composers.csv")
composers_birthplaces = set(composers_cities['birthplace'].dropna())
composers_deathplaces = set(composers_cities['place of death'].dropna())

In [39]:
publishing_places = set(publishers_cleaned['Place of publication'].dropna())

In [41]:
birthplaces_publication = composers_birthplaces.intersection(publishing_places)
print(birthplaces_publication)

deathplaces_publication = composers_deathplaces.intersection(publishing_places)
print(deathplaces_publication)

{'London', 'Padua', 'Paris', 'Bologna', 'Messina', 'Venice', 'Rome', 'Ferrara', 'Milan', 'Nuremberg', 'Florence', 'Naples', 'Palermo', 'Copenhagen'}
{'London', 'Cologne', 'Florence', 'Palermo', 'Copenhagen', 'Wolfenbüttel', 'Paris', 'Bologna', 'Venice', 'Frankfurt am Main', 'Padua', 'Genoa', 'Rome', 'Ferrara', 'Augsburg', 'Parma', 'Oxford', 'Dresden', 'Freiberg', 'Milan', 'Leipzig', 'Naples', 'Antwerp'}
